In [ ]:
%matplotlib notebook
from obspy import read
import numpy as np
import sys 
sys.path.append("/Users/Hilary/Documents/Github/Py-NnK/scan")
import os
import EQScanner
import EQ
import importlib
import matplotlib.pyplot as plt
import obspy
import trigger
import winsound
import time
import utm

In [ ]:
# Load Travel Time Model data from file
TTModel_P= np.load('save/Travel_Time_Model_P.npy')
TTModel_S= np.load('save/Travel_Time_Model_S.npy')

# Or try a slower velocity model...
#TTModel_P= np.load('save/Travel_Time_Model_P_-0.2km.npy')
#TTModel_S= np.load('save/Travel_Time_Model_S_P-0.2km_1.78.npy')
#TTModel_P= np.load('save/Travel_Time_Model_P_P-1.npy')
#TTModel_S= np.load('save/Travel_Time_Model_S_(P-1)178.npy')

In [ ]:
importlib.reload(EQScanner)

In [ ]:
# Generate all event location list for plotting them on the map later
rootdir = "/Users/Hilary/Documents/Github/Data/DataIceland"
for subdir,dirs,files in os.walk(rootdir): # Loop over the rootdir
       for file in dirs:
            #print(file)   # print to check the events
            read(os.path.join("/Users/Hilary/Documents/Github/Data/DataIceland",file,"ashE.Z.SAC"))

#### Read and process existing brightness matrices

In [ ]:
# Generate all event location list for plotting them on the map later
rootdir = "/Users/Hilary/Documents/Github/Data/DataIceland"
for subdir,dirs,files in os.walk(rootdir): # Loop over the rootdir
       for file in dirs:
            no_of_event =file
            for cf_type_l in [ ['CECM'],['ST*LT'], ['ST*LT','CECM'] ]:# [ ['CECM'],['ST*LT'], ['ST*LT','CECM'],['ST/LT'],['RP/LP'] ]:#['CECM'],['ST*LT'], ['ST*LT','CECM'], 
                cf_type_list =cf_type_l
                cf_type, cf_filename	= EQScanner.proper_cf_name(cf_type_list)
                if cf_type == 'CECM':
                    window_len = 2
                elif cf_type =='ST*LT':
                    window_len = 2
                elif cf_type =='ST*LT-CECM':
                    window_len = 2	
                elif cf_type == 'ST/LT':
                    window_len = 1   #3 cause smth dt=0.1
                elif cf_type =='RP/LP':
                    window_len = 1
                # Read the file
                C_temporal = np.load(
                    '\save\{}}_C_temporal_0.5km_0.1s_{}_EDT_w{}_scanPS.npy').format(
                    no_of_event,cf_filename,window_len)                
                

#### Scan PS

In [ ]:
# Generate all event location list for plotting them on the map later
rootdir = "/Users/Hilary/Documents/Github/Data/DataIceland"
for subdir,dirs,files in os.walk(rootdir): # Loop over the rootdir
       for file in dirs:
            no_of_event =file
            #print(file)   # print to check the events
            
            # Read a single event (with all the 18 stations)
            #no_of_event= 20101221012407
            #no_of_event= 20110222044937
            stm_original = read("/Users/Hilary/Documents/Github/Data/DataIceland/{}/*.sac".format(no_of_event))
            src_lalo=[stm_original[0].stats.sac.evla,stm_original[0].stats.sac.evlo]
            src_z=stm_original[0].stats.sac.evdp
            src_x=utm.from_latlon(src_lalo[0], src_lalo[1])[0]/100
            src_y=utm.from_latlon(src_lalo[0], src_lalo[1])[1]/100
            
            # Process and trim the raw trace
            trim_start=5
            trim_end= -225
            stm_processed=EQScanner.signal_processing(stm=stm_original,
                                        process=True,hp=5.0,lp=49.0, 
                                        trim= True, trim_start= trim_start, trim_end= trim_end, 
                                        plot=False)

            # For generating synthetic seismogram example (plot only 1 station)
            #stm_processed[:3].plot(size=(1200,400),outfile='save/figs/{}_syn_seis_ex.png'.format(no_of_event),format='png')

            for cf_type_l in [ ['CECM'],['ST/LT'],['RP/LP'] ]:#[ ['CECM'],['ST*LT'], ['ST*LT','CECM'],['ST/LT'],['RP/LP'] ]:#['CECM'],['ST*LT'], ['ST*LT','CECM'], 
                cf_type_list =cf_type_l
                cf_type,cf_E,cf_N,cf_Z= EQScanner.chracteristic_function(
                                                            stm_processed, cf_type_list= cf_type_list, 
                                                            plot= True, plot_size=(10,14), save_fig=True,
                                                            save_cf= False, no_of_event=no_of_event,linewidth=1)               
                
                plt.close()
                time.sleep(1)
                # Parameters for scanning event origin time
                tmin,tmax = 23,26                     # Range of possible origin time (in sec after the start time)
                dt = 0.1                             # Step for scanning specific time (in sec)
                #t_eo = 2.2

                # Parameters for scanning the hypocenter
                zmin,zmax = 0, 12        # Range of possible depth (in km)
                xmin,xmax = src_x-60, src_x+60               # E-W boundaries of the scanning area 
                                                     # (in Easting; omit the last 2 digits: ie. dx=100m)
                ymin,ymax = src_y-60, src_y+60             # S-N boundaries of the scanning area
                                                     # (in Northing; omit the last 2 digits: ie. dy=100m)
                dx = 5                               # x interval (ex. dx = 5 then resolusion = 500m)
                dy = 5                               # y interval (ex. dy = 5 then resolusion = 500m)
                dz = 0.5                             # z interval (ex. dx = 0.5 then resolusion = 0.5km)
                utm_zone = (27,'V')                  # UTM zone number of the scanning area (ex. (27,'V))
                method  = 'L1'                  # Option:'L2-Norm', 'EDT'
                #method  = 'EDT'                     # Option:'L2-Norm', 'EDT'

                #channel='Z'
                color_dict=  {'kri':'tab:blue','faf':'tab:orange','lat':'tab:green','sea':'tab:red',
                             'klv':'tab:purple','elb':'xkcd:sienna','ash':'tab:pink','iss':'tab:gray',
                             'hdv':'tab:olive','lsf':'tab:cyan','lhl':'xkcd:tomato','vog':'xkcd:teal',
                             'kas':'xkcd:green','blf':'xkcd:blue','bhr':'xkcd:orchid','hah':'xkcd:orangered',
                             'svh':'xkcd:khaki','has':'xkcd:grey'}

                # Scan to get the brightness matrices
                C_spatial, C_temporal    = EQScanner.SourceScan(   
                            stm_processed= stm_processed, 
                            Cf_Z=cf_Z, Cf_E=cf_E, Cf_N=cf_N, 
                            tmin= tmin, tmax= tmax, dt= dt,
                            zmin= zmin, zmax= zmax, dz= dz,
                            xmin= xmin, xmax= xmax, dx= dx,
                            ymin= ymin, ymax= ymax, dy= dy, 
                            utm_zone= utm_zone, TTModel_1= TTModel_P, TTModel_2=TTModel_S,
                            method=method, tmin_checker = False, 
                            save_C= True, no_of_event= no_of_event, cf_type_list=cf_type_list,
                            add_filename='_scanPS',alarm=False)
                time.sleep(1)

                C_s= C_spatial

                # Analyse to get the maximun brightness
                C_xy, C_zy, C_zx, coor_max_Cxy, coor_max_Czy, coor_max_Czx, imax_Cs= EQScanner.analyse_spatial(
                                            stm_processed= stm_processed, C_s= C_s,
                                            zmin= zmin, zmax= zmax, dz= dz,
                                            xmin= xmin, xmax= xmax, dx= dx,
                                            ymin= ymin, ymax= ymax, dy= dy)

                t_eo= EQScanner.analyse_temporal(
                    stm_processed= stm_processed, C_temporal= C_temporal, imax_Cs=imax_Cs, tmin=tmin)


                # Plot the Source Scanning result
                EQScanner.source_plot(    stm_processed=stm_processed,
                                C_xy=C_xy, C_zy=C_zy, C_zx=C_zx,
                                coor_max_Cxy=coor_max_Cxy, coor_max_Czy=coor_max_Czy, coor_max_Czx=coor_max_Czx,
                                dt= dt, method=method,cf_type_list=cf_type_list,
                                zmin= zmin, zmax= zmax, dz= dz,
                                xmin= xmin, xmax= xmax, dx= dx,
                                ymin= ymin, ymax= ymax, dy= dy,
                                plot_mp = False,# mp_scale = 0.1, C_6D=C_6D,
                                draw_rect=False, 
                                #rect_x=[4465,4485], rect_y=[70850,70880], rect_z= [0,50],
                                plot_ref= '--', figsize=[7,7.3], vmin=None, 
                                compare_dtdr=True, 
                                tmin= tmin, t_eo=t_eo, utm_zone= utm_zone, trim_start= trim_start,
                                save_fig=True, no_of_event= no_of_event, add_figname='_scanPS_nosmth')
                plt.close()
                # Smooth the brightness matrix
                C_s= EQScanner.smooth(      C_spatial,
                                            zmin= zmin, zmax= zmax, dz= dz,
                                            xmin= xmin, xmax= xmax, dx= dx,
                                            ymin= ymin, ymax= ymax, dy= dy)

                # Analyse to get the maximun brightness
                C_xy, C_zy, C_zx, coor_max_Cxy, coor_max_Czy, coor_max_Czx, imax_Cs= EQScanner.analyse_spatial(
                                            stm_processed= stm_processed, C_s= C_s,
                                            zmin= zmin, zmax= zmax, dz= dz,
                                            xmin= xmin, xmax= xmax, dx= dx,
                                            ymin= ymin, ymax= ymax, dy= dy)

                t_eo= EQScanner.analyse_temporal(
                    stm_processed= stm_processed, C_temporal= C_temporal, imax_Cs=imax_Cs, tmin=tmin)

                # Compare SSA result with SAC reference
                diff_in_t, diff_in_s = EQScanner.compare(	stm_processed=stm_processed,
                                    C_xy=C_xy, C_zy=C_zy, C_zx=C_zx,
                                    coor_max_Cxy=coor_max_Cxy, coor_max_Czy=coor_max_Czy, coor_max_Czx=coor_max_Czx,
                                    tmin= tmin, t_eo=t_eo, utm_zone= utm_zone, trim_start= trim_start,
                                    save = True, no_of_event=no_of_event, method=method, 
                                    add_title='_ScanPS_',cf_type=cf_type,first_cf_type='CECM', 
                                    filename='Results_real_slowPS_scanPS')   
                

                # Plot the Source Scanning result
                EQScanner.source_plot(    stm_processed=stm_processed,
                                C_xy=C_xy, C_zy=C_zy, C_zx=C_zx,
                                coor_max_Cxy=coor_max_Cxy, coor_max_Czy=coor_max_Czy, coor_max_Czx=coor_max_Czx,
                                dt= dt, method=method,cf_type_list=cf_type_list,
                                zmin= zmin, zmax= zmax, dz= dz,
                                xmin= xmin, xmax= xmax, dx= dx,
                                ymin= ymin, ymax= ymax, dy= dy,
                                plot_mp = False, #, mp_scale=0.1, C_4D=C_4D,
                                compare_dtdr=True, 
                                tmin= tmin, t_eo=t_eo, utm_zone= utm_zone, trim_start= trim_start,
                                draw_rect=False, 
                                #rect_x=[4465,4485], rect_y=[70850,70880], rect_z= [0,50],
                                plot_ref= '--', figsize=[7,7.3], vmin=None,
                                save_fig=True, no_of_event= no_of_event, add_figname='_scanPS_smth')
                plt.close()
                # Waveforms vs distances from the hypocenter (Source Scanning Result)
                EQScanner.waveform_plot_all(stm_processed= stm_processed, duration= 12, channel='Z', 
                                    TTModel= TTModel_P, TTModel_2=TTModel_S, 
                                    tmin= tmin, t_eo=t_eo,
                                    utm_zone= utm_zone, 
                                    figsize=[8,11], xy_label_size=20,  # nst18:[8,11], nst10:[8,8], nst5:[8,5]
                                    amp=0.6, color_dict=color_dict,
                                    save_fig= True, no_of_event=no_of_event, 
                                    method=method, cf_type_list=cf_type_list, add_figname='_scanPS_',
                                    title_size=24, add_v_sep=0.05,
                                    tt_marker='k+',tt_mk_size=6,tt_marker_sep=0.2) #syn:26,real: 6

                plt.close()
                # Waveforms vs distances from the hypocenter (Source Scanning Result)
                EQScanner.waveform_plot_all(stm_processed= stm_processed, duration= 12, channel='Z', 
                                    TTModel= TTModel_P, TTModel_2=TTModel_S, 
                                    trim_start= trim_start,
                                    utm_zone= utm_zone, 
                                    figsize=[8,11], xy_label_size=20,  # nst18:[8,11], nst10:[8,8], nst5:[8,5]
                                    amp=0.6, color_dict=color_dict,
                                    save_fig= True, no_of_event=no_of_event, 
                                    method='SAC', cf_type_list=cf_type_list, add_figname='_SAC_',
                                    title_size=24, add_v_sep=0.05,
                                    tt_marker='k+',tt_mk_size=6,tt_marker_sep=0.2) #syn:26,real: 6
                plt.close()
                time.sleep(1)
            winsound.Beep(523, 1000)
            winsound.Beep(587, 500)
            winsound.Beep(523, 500)

#### Scan P

In [ ]:
# Generate all event location list for plotting them on the map later
rootdir = "/Users/Hilary/Documents/Github/Data/DataIceland"
for subdir,dirs,files in os.walk(rootdir): # Loop over the rootdir
       for file in dirs:
            no_of_event =file
            #print(file)   # print to check the events
            
            # Read a single event (with all the 18 stations)
            #no_of_event= 20101221012407
            #no_of_event= 20110222044937
            stm_original = read("/Users/Hilary/Documents/Github/Data/DataIceland/{}/*.sac".format(no_of_event))
            src_lalo=[stm_original[0].stats.sac.evla,stm_original[0].stats.sac.evlo]
            src_z=stm_original[0].stats.sac.evdp
            src_x=utm.from_latlon(src_lalo[0], src_lalo[1])[0]/100
            src_y=utm.from_latlon(src_lalo[0], src_lalo[1])[1]/100
            
            # Process and trim the raw trace
            trim_start=5
            trim_end= -225
            stm_processed=EQScanner.signal_processing(stm=stm_original,
                                        process=True,hp=5.0,lp=49.0, 
                                        trim= True, trim_start= trim_start, trim_end= trim_end, 
                                        plot=False)

            # For generating synthetic seismogram example (plot only 1 station)
            #stm_processed[:3].plot(size=(1200,400),outfile='save/figs/{}_syn_seis_ex.png'.format(no_of_event),format='png')

            for cf_type_l in [ ['CECM'],['ST*LT'], ['ST*LT','CECM'] ]:#[ ['CECM'],['ST*LT'], ['ST*LT','CECM'],['ST/LT'],['RP/LP'] ]:#['CECM'],['ST*LT'], ['ST*LT','CECM'], 
                cf_type_list =cf_type_l
                cf_type,cf_E,cf_N,cf_Z= EQScanner.chracteristic_function(
                                                            stm_processed, cf_type_list= cf_type_list, 
                                                            plot= True, plot_size=(10,14), save_fig=False,
                                                            save_cf= False, no_of_event=no_of_event,linewidth=1)               
                
                plt.close()
                time.sleep(1)
                # Parameters for scanning event origin time
                tmin,tmax = 23,26                     # Range of possible origin time (in sec after the start time)
                dt = 0.05                             # Step for scanning specific time (in sec)
                #t_eo = 2.2

                # Parameters for scanning the hypocenter
                zmin,zmax = 0, 12        # Range of possible depth (in km)
                xmin,xmax = src_x-60, src_x+60               # E-W boundaries of the scanning area 
                                                     # (in Easting; omit the last 2 digits: ie. dx=100m)
                ymin,ymax = src_y-60, src_y+60             # S-N boundaries of the scanning area
                                                     # (in Northing; omit the last 2 digits: ie. dy=100m)
                dx = 5                               # x interval (ex. dx = 5 then resolusion = 500m)
                dy = 5                               # y interval (ex. dy = 5 then resolusion = 500m)
                dz = 0.5                             # z interval (ex. dx = 0.5 then resolusion = 0.5km)
                utm_zone = (27,'V')                  # UTM zone number of the scanning area (ex. (27,'V))
                method  = 'L1'                  # Option:'L2-Norm', 'EDT'
                #method  = 'EDT'                     # Option:'L2-Norm', 'EDT'

                #channel='Z'
                color_dict=  {'kri':'tab:blue','faf':'tab:orange','lat':'tab:green','sea':'tab:red',
                             'klv':'tab:purple','elb':'xkcd:sienna','ash':'tab:pink','iss':'tab:gray',
                             'hdv':'tab:olive','lsf':'tab:cyan','lhl':'xkcd:tomato','vog':'xkcd:teal',
                             'kas':'xkcd:green','blf':'xkcd:blue','bhr':'xkcd:orchid','hah':'xkcd:orangered',
                             'svh':'xkcd:khaki','has':'xkcd:grey'}

                # Scan to get the brightness matrices
                C_spatial, C_temporal    = EQScanner.SourceScan_Phase(   
                            stm_processed= stm_processed, 
                            Cf_Z=cf_Z,# Cf_E=cf_E, Cf_N=cf_N, 
                            tmin= tmin, tmax= tmax, dt= dt,
                            zmin= zmin, zmax= zmax, dz= dz,
                            xmin= xmin, xmax= xmax, dx= dx,
                            ymin= ymin, ymax= ymax, dy= dy, 
                            utm_zone= utm_zone, TTModel_1= TTModel_P, TTModel_2=None,
                            method=method, tmin_checker = False, 
                            save_C= True, no_of_event= no_of_event, cf_type_list=cf_type_list,
                            add_filename='_scanP',alarm=False)
                time.sleep(1)

                C_s= C_spatial

                # Analyse to get the maximun brightness
                C_xy, C_zy, C_zx, coor_max_Cxy, coor_max_Czy, coor_max_Czx, imax_Cs= EQScanner.analyse_spatial(
                                            stm_processed= stm_processed, C_s= C_s,
                                            zmin= zmin, zmax= zmax, dz= dz,
                                            xmin= xmin, xmax= xmax, dx= dx,
                                            ymin= ymin, ymax= ymax, dy= dy)

                t_eo= EQScanner.analyse_temporal(
                    stm_processed= stm_processed, C_temporal= C_temporal, imax_Cs=imax_Cs, tmin=tmin)


                # Plot the Source Scanning result
                EQScanner.source_plot(    stm_processed=stm_processed,
                                C_xy=C_xy, C_zy=C_zy, C_zx=C_zx,
                                coor_max_Cxy=coor_max_Cxy, coor_max_Czy=coor_max_Czy, coor_max_Czx=coor_max_Czx,
                                dt= dt, method=method,cf_type_list=cf_type_list,
                                zmin= zmin, zmax= zmax, dz= dz,
                                xmin= xmin, xmax= xmax, dx= dx,
                                ymin= ymin, ymax= ymax, dy= dy,
                                plot_mp = False,# mp_scale = 0.1, C_6D=C_6D,
                                draw_rect=False, 
                                #rect_x=[4465,4485], rect_y=[70850,70880], rect_z= [0,50],
                                plot_ref= '--', figsize=[7,7.3], vmin=None, 
                                compare_dtdr=True, 
                                tmin= tmin, t_eo=t_eo, utm_zone= utm_zone, trim_start= trim_start,
                                save_fig=True, no_of_event= no_of_event, add_figname='_scanP_nosmth')
                plt.close()
                # Smooth the brightness matrix
                C_s= EQScanner.smooth(      C_spatial,
                                            zmin= zmin, zmax= zmax, dz= dz,
                                            xmin= xmin, xmax= xmax, dx= dx,
                                            ymin= ymin, ymax= ymax, dy= dy)

                # Analyse to get the maximun brightness
                C_xy, C_zy, C_zx, coor_max_Cxy, coor_max_Czy, coor_max_Czx, imax_Cs= EQScanner.analyse_spatial(
                                            stm_processed= stm_processed, C_s= C_s,
                                            zmin= zmin, zmax= zmax, dz= dz,
                                            xmin= xmin, xmax= xmax, dx= dx,
                                            ymin= ymin, ymax= ymax, dy= dy)

                t_eo= EQScanner.analyse_temporal(
                    stm_processed= stm_processed, C_temporal= C_temporal, imax_Cs=imax_Cs, tmin=tmin)

                # Compare SSA result with SAC reference
                diff_in_t, diff_in_s = EQScanner.compare(	stm_processed=stm_processed,
                                    C_xy=C_xy, C_zy=C_zy, C_zx=C_zx,
                                    coor_max_Cxy=coor_max_Cxy, coor_max_Czy=coor_max_Czy, coor_max_Czx=coor_max_Czx,
                                    tmin= tmin, t_eo=t_eo, utm_zone= utm_zone, trim_start= trim_start,
                                    save = True, no_of_event=no_of_event, 
                                    method=method, add_title='_ScanP_',
                                    cf_type=cf_type,first_cf_type='CECM', filename='Results_res')   
                

                # Plot the Source Scanning result
                EQScanner.source_plot(    stm_processed=stm_processed,
                                C_xy=C_xy, C_zy=C_zy, C_zx=C_zx,
                                coor_max_Cxy=coor_max_Cxy, coor_max_Czy=coor_max_Czy, coor_max_Czx=coor_max_Czx,
                                dt= dt, method=method,cf_type_list=cf_type_list,
                                zmin= zmin, zmax= zmax, dz= dz,
                                xmin= xmin, xmax= xmax, dx= dx,
                                ymin= ymin, ymax= ymax, dy= dy,
                                plot_mp = False, #, mp_scale=0.1, C_4D=C_4D,
                                compare_dtdr=True, 
                                tmin= tmin, t_eo=t_eo, utm_zone= utm_zone, trim_start= trim_start,
                                draw_rect=False, 
                                #rect_x=[4465,4485], rect_y=[70850,70880], rect_z= [0,50],
                                plot_ref= '--', figsize=[7,7.3], vmin=None,
                                save_fig=True, no_of_event= no_of_event, add_figname='_scanP_smth')
                plt.close()
                # Waveforms vs distances from the hypocenter (Source Scanning Result)
                EQScanner.waveform_plot_all(stm_processed= stm_processed, duration= 12, channel='Z', 
                                    TTModel= TTModel_P, TTModel_2=TTModel_S, 
                                    tmin= tmin, t_eo=t_eo,
                                    utm_zone= utm_zone, 
                                    figsize=[8,11], xy_label_size=20,  # nst18:[8,11], nst10:[8,8], nst5:[8,5]
                                    amp=0.6, color_dict=color_dict,
                                    save_fig= True, no_of_event=no_of_event, 
                                    method=method, cf_type_list=cf_type_list, add_figname='_scanP_',
                                    title_size=24, add_v_sep=0.05,
                                    tt_marker='k+',tt_mk_size=6,tt_marker_sep=0.2) #syn:26,real: 6

                plt.close()
                # Waveforms vs distances from the hypocenter (Source Scanning Result)
                EQScanner.waveform_plot_all(stm_processed= stm_processed, duration= 12, channel='Z', 
                                    TTModel= TTModel_P, TTModel_2=TTModel_S, 
                                    trim_start= trim_start,
                                    utm_zone= utm_zone, 
                                    figsize=[8,11], xy_label_size=20,  # nst18:[8,11], nst10:[8,8], nst5:[8,5]
                                    amp=0.6, color_dict=color_dict,
                                    save_fig= True, no_of_event=no_of_event, 
                                    method='SAC', cf_type_list=cf_type_list, add_figname='_SAC_',
                                    title_size=24, add_v_sep=0.05,
                                    tt_marker='k+',tt_mk_size=6,tt_marker_sep=0.2) #syn:26,real: 6
                plt.close()
                time.sleep(1)
winsound.Beep(523, 1000)
winsound.Beep(587, 500)
winsound.Beep(523, 500)

In [ ]:
# Waveforms vs distances from the hypocenter (SAC Attribute)
EQScanner.waveform_plot_SAC(stm_processed= stm_processed, duration=6, channel='Z',
                            TTModel=TTModel,
                            trim_start=trim_start,
                            figsize=[7,11],amp=0.6)

In [ ]:
importlib.reload(EQScanner)

In [ ]:
cf_type ='ST*LT'

Qualitative:

1. 

In [ ]:
# Optional: Select only closed stations
st_info_list = EQScanner.st_basic_info(stm=stm, channel='Z', src_lalo=src_lalo, src_z=src_z ,TTModel=TTModel_P)
st_info_list=sorted(st_info_list, key=lambda x: x[1]) # Sorted by distance

# Optional: Select only closed stations
nst=4
no_of_event = no_of_event + '_nst_far{}'.format(nst)
st_selected_list = st_info_list[(18-nst):].copy()
stm_selected = obspy.Stream()
for tr in stm:
    for st_selected in st_selected_list:
        if tr.stats.station[:3] == st_selected[0][:3]:
            stm_selected.append(tr)